<a href="https://colab.research.google.com/github/bhatsbharath/generative_ai_agents/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ChatBOT

This code defines a function chat_with_ollama that sends a prompt to a local Ollama language model API (llama3) and returns the generated response.

It uses the requests library to POST the prompt to http://localhost:11434/api/generate.

The stream parameter is set to False to get the full response at once.

If the request is successful, it returns the generated text; otherwise, it returns an error message.

The example shows how to ask the model a question about multi-agent systems in generative AI.

In [ ]:
import requests

def chat_with_ollama(prompt, model="llama3"):
    url = "http://localhost:11434/api/generate"
    payload = {
        "model": model,
        "prompt": prompt,
        "stream": False  # stream = True if you want to process partial responses
    }

    response = requests.post(url, json=payload)
    if response.status_code == 200:
        return response.json()['response']
    else:
        return f"Error: {response.status_code} - {response.text}"

# Example usage
response = chat_with_ollama("What is multi agent system in generative ai.")
print(response)


This code defines a function generate_response that sends a prompt to a local language model API (llama3) and gets a generated text response.

It uses the requests library to POST the prompt with parameters like model and temperature.

If successful, it returns the generated text; otherwise, it shows an error.

The script tests two temperature values (0.2 and 1.2) to show how temperature affects creativity:

Lower temperature → more predictable responses.

Higher temperature → more creative and varied responses.

In [ ]:
import requests

def generate_response(prompt, model="llama3", temperature=0.7):
    url = "http://localhost:11434/api/generate"
    payload = {
        "model": model,
        "prompt": prompt,
        "temperature": temperature,
        "stream": False
    }

    response = requests.post(url, json=payload)
    if response.status_code == 200:
        return response.json()["response"]
    else:
        return f"Error: {response.status_code} - {response.text}"


prompt = "Write a short sci-fi story in 4-5 sentences about a robot that becomes self-aware.."

# Try different temperatures
for temp in [0.2, 1.2]:
    print(f"\n--- Temperature: {temp} ---")
    print(generate_response(prompt, temperature=temp))

# temperature=0.2: more predictable, safe names like “AIVA” or “NeoAI”.
# temperature=1.2: more creative, unusual names like “QuantiNova” or “MindPhantom”.

In [ ]:
# Zero-Shot Prompting
prompt = "Translate 'I like pizza' into French."
print(generate_response(prompt))

In [ ]:
# One-Shot Prompting
prompt = """Translate the following sentences into French:

English: Good morning
French: Bonjour

English: I like pizza
French:"""
print(generate_response(prompt))


In [ ]:
#Few shot prompting
prompt = """Translate the following sentences into French:

English: Good morning
French: Bonjour

English: How are you?
French: Comment ça va ?

English: I like pizza
French:"""
print(generate_response(prompt))


In [ ]:
import ollama

response = ollama.chat(
    model='llama3.2-vision',
    messages=[{
        'role': 'user',
        'content': 'What is in this image?',
        'images': ['TestImage.png']
    }]
)

print(response)

In [ ]:
from langchain_community.chat_models import ChatOllama
from langchain.prompts import PromptTemplate

llm = ChatOllama(
	model="llama3.2",
	temperature=0.7,
	top_p=0.9,
	num_predict=256,
	repeat_penalty=1.1
)

# Standard prompt
standard_prompt = PromptTemplate(
    input_variables=["question"],
    template="Answer the following question concisely: {question}."
)

# Chain of Thought prompt
cot_prompt = PromptTemplate(
    input_variables=["question"],
    template="Answer the following question step by step concisely: {question}"
)

# Create chains
standard_chain = standard_prompt | llm
cot_chain = cot_prompt | llm

# Example question
question = "If a train travels 120 km in 2 hours, what is its average speed in km/h?"

# Get responses
standard_response = standard_chain.invoke(question).content
cot_response = cot_chain.invoke(question).content

print("Standard Response:")
print(standard_response)
print("\nChain of Thought Response:")
print(cot_response)